In [29]:
import sys
import os
import copy
sys.path.append('../')
from model import NextWordPredictor, train, test

In [3]:
import pandas as pd
import torch
import torchtext.functional as F

In [4]:
train_d = pd.read_csv('../data/stack_overflow/train.csv')
test_d = pd.read_csv('../data/stack_overflow/test.csv')
val_d = pd.read_csv('../data/stack_overflow/val.csv')

In [5]:
train_sentence = train_d['text'][0]
test_sentence = test_d['text'][0]
val_sentence = val_d['text'][0]

In [6]:
words1 = train_sentence.split()
words2 = test_sentence.split()

In [7]:
vocab = set(words1 + words2)

In [8]:
word_to_index = { word: i for i, word in enumerate(vocab) }
index_to_word = { i: word for word, i in word_to_index.items() }

In [9]:
vocab_size = len(vocab)
vocab_size

7843

In [10]:
indices1 = [word_to_index[word] for word in words1]
indices2 = [word_to_index[word] for word in words2]

In [11]:
train_data = torch.tensor(indices1 + indices2, dtype=torch.long).view(1, -1)  # Combine both sentences for training
train_targets = torch.tensor(indices1 + indices2, dtype=torch.long).view(1, -1)  # Targets are next words

In [12]:
test_data = torch.tensor(indices2, dtype=torch.long).view(1, -1)  # Test sequence
test_targets = torch.tensor(indices2, dtype=torch.long).view(1, -1)  # Test targets

In [13]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_data, train_targets), batch_size=1)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_data, test_targets), batch_size=1)

In [14]:
# Model parameters
embedding_dim = 10
hidden_dim = 10
epochs = 10

In [15]:
import torch.nn as nn
import torch.optim as optim

In [16]:
# Initialize model, criterion, and optimizer
model = NextWordPredictor(vocab_size, embedding_dim, hidden_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [17]:
train(model, train_loader, criterion, optimizer, epochs)

/home/nadun/Documents/projects/flower/.venv/lib/python3.10/site-packages/torch/autograd/__init__.py:266: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 1/10, Loss: 9.00102424621582
Epoch 2/10, Loss: 8.961320877075195
Epoch 3/10, Loss: 8.922078132629395
Epoch 4/10, Loss: 8.881683349609375
Epoch 5/10, Loss: 8.838608741760254
Epoch 6/10, Loss: 8.791695594787598
Epoch 7/10, Loss: 8.739978790283203
Epoch 8/10, Loss: 8.682480812072754
Epoch 9/10, Loss: 8.618148803710938
Epoch 10/10, Loss: 8.545866966247559


In [18]:
test(model, test_loader, criterion)

Test Loss: 8.459348678588867


In [23]:
path = '../results/test'
if not os.path.exists(path):
    os.makedirs(path)

torch.save(model.state_dict(), f'{path}/model.pth')

In [31]:
model_path = f'{path}/model.pth'

if os.path.exists(model_path):
    new_model = copy.deepcopy(model)
    new_model.load_state_dict(torch.load(model_path))
    print(new_model)

NextWordPredictor(
  (embedding): Embedding(7843, 10)
  (rnn): LSTM(10, 10, batch_first=True)
  (fc): Linear(in_features=10, out_features=7843, bias=True)
)


In [32]:
test(new_model, test_loader, criterion)

Test Loss: 8.459348678588867
